# POSIBLES MEJORAS

Se descirbirán posibles mejoras para aplicar al proyecto base

### NORMALIZACIÓN
Objetivo: Hacer que las características dependan menos del volumen de grabación.

In [1]:
# --- Mejora 1: Normalización ---
import librosa
import librosa.util # Importar librosa.util
import numpy as np

def compute_vocal_fingerprint_normalized(audio_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=sr)

    # --- NUEVO ---
    # Normalizar la señal para que tenga un rango estándar (-1 a 1 aprox)
    y = librosa.util.normalize(y)
    # -----------

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    huella = np.mean(mfccs, axis=1) # Seguimos promediando por ahora
    return huella


## Reducción de Ruido:

### Libería Pedalboard

En el siguiente [articulo](https://medium.com/@joshiprerak123/transform-your-audio-denoise-and-enhance-sound-quality-with-python-using-pedalboard-24da7c1df042) se describe como hacerlo, PREGUNTAR AL PROFESOR si se puede utilizar.

### scipy.signal 
Si no se peude mirar en este [enlace](https://docs.scipy.org/doc/scipy/reference/signal.html) y pdoer aplicar el filtorde Weiner (para su aplicación hay que estimar el ruido)

## Mejor cálculo de huellas vocales

Mirar la [documentación](https://librosa.org/doc/latest/feature.html) de librosa.feature.delta para calcular deltas (velocidad) y delta-deltas (aceleración) de los MFCCs.

In [ ]:
import librosa
import librosa.util
import numpy as np

def compute_vocal_fingerprint_deltas_mean(audio_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=sr)
    y = librosa.util.normalize(y)

    # --- MODIFICADO ---
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Calcular deltas (orden 1)
    delta_mfccs = librosa.feature.delta(mfccs)
    # Calcular delta-deltas (orden 2)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)

    # Combinar las características (ahora tenemos 3*n_mfcc filas)
    combined_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))

    # Promediar todas juntas
    huella_vocal = np.mean(combined_features, axis=1)
    # ----------------
    # La huella ahora tiene n_mfcc * 3 elementos
    return huella_vocal


## Uso de mejores modelos

### Métricas de Distancia Alternativas
[SciPy Spatial Distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) (scipy.spatial.distance): Para usar distancias como Coseno (cosine) o Manhattan (cityblock) en lugar de Euclidiana.



In [ ]:
# --- Comparación con Distancia Coseno ---
from scipy.spatial.distance import cosine # Importar cosine

def compare_vocal_fingerprints_cosine(x, y, threshold=0.4): # UMBRAL DIFERENTE!
    # x e y son las huellas obtenidas de alguna de las funciones anteriores

    # La distancia coseno va de 0 (idénticos) a 2 (opuestos)
    distance = cosine(x, y)
    # ----------------

    # La decisión es la misma: distancia < umbral, pero el umbral es distinto
    # Umbrales típicos para coseno suelen ser < 1 (ej. 0.1 a 0.6)
    return distance < threshold, distance

### Modelos Probabilísticos
[Scikit-learn Docs](https://scikit-learn.org/stable/modules/mixture.html) - Gaussian Mixture Models: Para implementar GMMs (sklearn.mixture.GaussianMixture) y usarlos para modelar la distribución de características de cada locutor.

### Redes Neuronales
[Tutorial Keras](https://keras.io/examples/vision/siamese_network/) - Siamese Networks: Aplicar el concepto de redes siamesas para aprender una métrica de distancia sobre las huellas vocales.